In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import json
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split, DataLoader, Dataset

import torch
from torchvision import datasets, transforms

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q datasets jiwer

In [2]:

#! mkdir "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\BallerTV"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))
     

In [3]:
nums = []
num = 0
anno_dict = {}

for path in tqdm(anno_pathses):

    # Load the XML file
    tree = ET.parse(path)

    # Get the root element of the XML tree
    root = tree.getroot()

    # Iterate over the child elements of the root element
    for image in root.findall('image'):
        # Get the image ID and file name
        image_id = image.get('id')
        file_name = image.get('name')
        
        # Iterate over all box elements in the image
        for box in image.findall('box'):
            image = Image.open(file_name)

            # Get the box label, coordinates, and attributes
            label = box.get('label')
            
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + str(num) + 'example.jpg'
                anno_dict['label'] = int(number)
                anno_dict['path'] = res_path 

                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_pathses)
df.to_csv("anno.csv")



 24%|██▍       | 25/105 [00:52<02:46,  2.08s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'BallerTV/basketball/2_processed/10-08-2022/frames/7098_94483_1658680618_raw/000020.jpeg'

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

#### В этом блоке мы смотрим в ручном режиме как решает нашу задачу модель из коробки

In [10]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Номер на футболке: 5


In [28]:
transform = transforms.Compose(
        [transforms.RandomResizedCrop(224), 
         transforms.ToTensor()])

In [ ]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [29]:
root_path = 'crops'
batch_size = 32

dataset = datasets.ImageFolder(root=root_path, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [30]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(test_dataset))

Number of training examples: 3679
Number of validation examples: 920


In [31]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [32]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=1000,
    eval_steps=200,
)

In [33]:
from datasets import load_metric

cer_metric = load_metric("cer")

KeyboardInterrupt: 

In [34]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [35]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=default_data_collator,
)
trainer.train()

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TypeError: vars() argument must have __dict__ attribute